'''

Author: Jayesh Patil

Date: 21-11-2024

Title: Python program to perform Gen AI tasks Summarizing and Transforming using Gemini API

'''

Importing Libraries

In [60]:
import os
import csv
import google.generativeai as genai
from dotenv import load_dotenv

Loading Env variable

In [61]:
load_dotenv()


True

Config Gemini AI

In [62]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))


In [63]:
generation_config = {
  "temperature": 0.9,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

In [64]:
def read_emails(file_path, delimiter="END"):
    """
    Description: 
        This function reads the email from the text file
    Parameters:
        file_path: Path of txt file
        delimiter: delimiter
    Return:
        list: list of emails
    """
    with open(file_path, 'r') as file:
        emails = file.read().split(delimiter)
    return [email.strip() for email in emails if email.strip()]  

In [65]:
def extract_email_info(email):
    """
    Description: 
        This Function extracts sender, receiver, and body from email
    Parameters:
        email
    Return:
        sender, receiver, and body (str)
    """
    lines = email.split("\n")
    sender = receiver = body = ""
    
    for line in lines:
        if line.startswith("From:"):
            sender = line.split(":")[1].strip()
        elif line.startswith("To:"):
            receiver = line.split(":")[1].strip()
        elif not line.startswith("Subject:") and not line.startswith("From:") and not line.startswith("To:"):
            body += line.strip() + " "
    
    return sender, receiver, body.strip()

Summarizing Task

In [66]:
def summarize_email(body, chat_session):
    """
    Description: 
        This Function summarizes email body using the Gemini model
    Parameters:
        body: body of email
        chat_session: gemini ai chat session
    Return:
        response(str): summarized response
    """
    response = chat_session.send_message(f"Summarize the following email: {body}")
    return response.text

In [67]:
def translate_email(text, chat_session):
    """
    Description: 
        This Function translates text using the Gemini model
    Parameters:
        body: body of email
        chat_session: gemini ai chat session
    Return:
        response(str): summarized response
    """
    response = chat_session.send_message(f"Translate the following text to Hindi: {text}")
    return response.text

In [68]:
def save_to_csv(data, csv_file):
    """
    Description: 
        This function save the response in csv file
    Parameters:
        data: data to be saved in csv
        csv_file: csv file name/path
    """
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Sender", "Receiver", "Summary (EN)", "Summary (Hindi)"])
        writer.writerows(data)

In [69]:
def process_emails(file_path, csv_file, chat_session, delimiter="END"):
    """
    Processes emails by summarizing and translating their content, and saves results to a CSV file.
    
    Parameters:
        file_path (str): Path to the email text file.
        csv_file (str): File path for the output CSV file.
        chat_session: A Gemini AI chat session object.
        delimiter (str): Delimiter used to separate emails in the file.
    """
    emails = read_emails(file_path, delimiter)
    if not emails:
        print("No emails to process.")
        return
    
    data = []
    for email in emails:
        sender, receiver, body = extract_email_info(email)
        if body:
            summary_en = summarize_email(body, chat_session)
            summary_hi = translate_email(summary_en, chat_session)
            data.append([sender, receiver, summary_en, summary_hi])
        else:
            print(f"Skipped email from {sender} to {receiver} due to empty body.")
    
    save_to_csv(data, csv_file)

In [70]:
def main():
    chat_session = model.start_chat(
        history=[]
    )
    process_emails('email.txt', 'email_pro.csv', chat_session)


if __name__ == '__main__':
    main()